In [ ]:
# importing necessary libraries 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision 
from torchvision import transforms, datasets, models 
from torch.nn import functional as F 
from PIL import Image
import pandas as pd 
import numpy as np
import tensorflow as tf
import os
import sys
import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
import cv2
import json 
from tqdm import tqdm
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from skimage.transform import rotate, AffineTransform
import random
from scipy import ndimage
import openslide
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from collections import Counter
import shutil 
import gc 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix 
import math 
import seaborn as sns 
from collections import defaultdict 
import matplotlib 
from sklearn.cluster import DBSCAN 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
patch_size = 0 
model_weights_pth = 'coreseg-weights.pth' 

**PREPARING CLINICAL DATA**

In [ ]:
# reading patients' data to obtain patient ids (her2+) 

df = pd.read_csv('HER2.csv')     # the path can be replaced by TNBC's path 
her2_patient_ids = list(df['Unique ID']) 
len(her2_patient_ids) 

**OBTAINING WSI DIMENSIONS**

In [ ]:
# loading dimensions of all wsis in a dictionary 

dimensions_dict = {} 

dim_df = pd.read_csv('/kaggle/input/impress-complete-patches-and-model-weights/wsi_dimensions.csv') 
dim_wsis = dim_df['WSI Id'].to_list() 
widths = dim_df['Width'].to_list() 
heights = dim_df['Height'].to_list() 

for i in range(len(dim_wsis)): 
    dimensions_dict[dim_wsis[i]] = (widths[i], heights[i]) 

In [ ]:
# Function to obtain patches and their labels against annotation matrices 

def get_all_patches_info(df): 

    wsi_nos = df['WSI No.'].to_list() 
    x_pixels = df['X Pixel'].to_list() 
    y_pixels = df['Y Pixel'].to_list() 
    updated_labels = df['Exact Label'].to_list() 

    patch_dir = '/patch_dir' 
    
    patch_paths = [f"{patch_dir}0{str(wsi_nos[i])}_{str(x_pixels[i])}_{str(y_pixels[i])}.jpg" if len(str(wsi_nos[i])) == 2 else f"{patch_dir}{str(wsi_nos[i])}_{str(x_pixels[i])}_{str(y_pixels[i])}.jpg" for i in range(len(wsi_nos))] 

    return wsi_nos, patch_paths, updated_labels, x_pixels, y_pixels 

In [ ]:
# patches dataframes 

her2_patch_info_df = pd.read_csv(f'/kaggle/input/impress-complete-patches-and-model-weights/all_wsi_her2_patches_{patch_size}_info.csv') 

her2_patch_info_df = her2_patch_info_df[her2_patch_info_df['WSI No.'].isin(her2_patient_ids)] 

In [ ]:
# obtaining patch infos for all wsis 

her2_wsi_nos, her2_patch_paths, her2_updated_labels, her2_x_pixels, her2_y_pixels = get_all_patches_info(her2_patch_info_df) 

wsi_nos = tnbc_wsi_nos  
patch_paths = tnbc_patch_paths 
updated_labels_temp = tnbc_updated_labels 
x_pixels = tnbc_x_pixels 
y_pixels = tnbc_y_pixels 

In [ ]:
# Encode with binary class labels

label_to_onehot = {
    'Background': 0 
}

# Mark 'Background' as 0, 'Foreground' label as 1
updated_labels = [label_to_onehot.get(label, 1) for label in updated_labels_temp] 

**CLASS DISTRIBUTION** 

In [ ]:
import collections

# Count the occurrences of each label (0 or 1)
label_counts = collections.Counter(updated_labels)

# Plot
plt.figure(figsize=(6, 4))
plt.bar(label_counts.keys(), label_counts.values(), color=['skyblue', 'salmon'])
plt.xticks([0, 1], ['Background (0)', 'Dense (1)'])
plt.ylabel('Count')
plt.title('Distribution of Updated Labels')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

**CUSTOM DATASET & DATA LOADER FOR TESTING** 

In [ ]:
from torchvision.transforms import ToTensor

class CustomDataset(Dataset):
    def __init__(self, patches, labels):
        assert len(patches) == len(labels), "Mismatch in number of patches and labels"
        self.patches = patches
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Check for valid index range
        if idx >= len(self.labels) or idx >= len(self.patches):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self)}")

        patch = Image.open(self.patches[idx]).convert("RGB")  # Adjust this if `patches` are file paths
        patch = patch.resize((224, 224))  
        patch = ToTensor()(patch)
        patch = nn.functional.normalize(patch, dim=0, p=2)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return patch, label

In [ ]:
# Create instances of custom datasets

test_dataset = CustomDataset(patch_paths, updated_labels) 

In [ ]:
batch_size = 64 

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 4) 

**MODEL (USING IMPORTED WEIGHTS FROM LAN BACH B)** 

In [ ]:
from torch.nn import functional as F
import torchvision.models as models

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
        self.base = models.vit_b_32(weights='DEFAULT')

        # freeze model weights
        for param in self.base.parameters():
            param.requires_grad = False
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
        
        # freeze head weights
        for param in self.head.parameters():
            param.requires_grad = False
            
    def forward(self,X):
            X = self.base(X)
            X = self.flatten(X)
            X = self.head(X)
            return X, F.sigmoid(X)
    
pos_weight = torch.tensor([0.0])
pos_weight = pos_weight.to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
model = Pipeline(1)
#model = nn.DataParallel(model) 

In [ ]:
# importing weights from LAN ran on BACH B 224 patches 

model.load_state_dict(torch.load(model_weights_pth)) 

In [ ]:
model = model.to(device) 

#criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.0) 

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1 

**EVALUATING** 

In [ ]:
model.eval()

test_loss = 0
correct = 0
total = 0
y_true_test = []
y_pred_test = []
y_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        # Move data to GPU if available
        images = images.to(device)
        labels = labels.to(device)
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        test_loss += loss.item()  # L+=l.item()
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true_test.extend(labels.cpu().detach().numpy())
        y_pred_test.extend(predicted.cpu().detach().numpy())
        y_probs.extend(outputs_with_sigmoid.cpu().detach().numpy())

In [ ]:
test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

# Calculate ROC-AUC for test set
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs)
roc_auc_test = auc(fpr_test, tpr_test)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.2f}%')
print(f'Test ROC-AUC: {roc_auc_test:.4f}')

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_true_test, y_pred_test)) 

In [ ]:
# ROC curve for the test set
plt.figure(figsize=(4, 4))
plt.plot(fpr_test, tpr_test, label=f'Test ROC Curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
plt.legend()
plt.show()

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true_test, y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Patch Level Confusion Matrix')
plt.show() 

**PREDICTION AND PROBABILITY MAPS** 

In [ ]:
matplotlib.use('Agg')  # Use non-interactive backend for speed
from mpl_toolkits.axes_grid1 import make_axes_locatable

num_classes = 2
save_dir_binary = "/prediction_maps"
os.makedirs(save_dir_binary, exist_ok=True)
save_dir_probs = "/probability_maps"
os.makedirs(save_dir_probs, exist_ok=True)

curr = prev = wsi_nos[0]
width, height = dimensions_dict[curr]
width = width // patch_size 
height = height // patch_size 
curr_ann_map_binary = np.zeros((height, width), dtype=np.uint8) 
curr_ann_map_probs = np.zeros((height, width), dtype=np.float16) 
kernel = np.ones((2, 2), np.uint8)  # adjust for gap size

all_ann_binary_maps = [] 
all_ann_probs_maps = [] 

for i in tqdm(range(len(y_true_test))):
    curr = wsi_nos[i]

    if curr != prev:

        # BINARY 
        # Apply morphological gap closing
        curr_ann_map_binary = cv2.morphologyEx(curr_ann_map_binary, cv2.MORPH_CLOSE, kernel)
        # Save the accumulated annotation matrix for the previous WSI 
        all_ann_binary_maps.append(curr_ann_map_binary) 
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = plt.get_cmap('PuRd', num_classes)
        im = ax.imshow(curr_ann_map_binary, cmap=cmap, interpolation='nearest', vmin=0, vmax=num_classes - 1)
        ax.set_title(f'Prediction Map for WSI {prev}')
        ax.axis('off')
        # Shrink the colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="1.5%", pad=0.03)
        plt.colorbar(im, cax=cax, ticks=range(num_classes), label='Class Index')
        plt.savefig(os.path.join(save_dir_binary, f"{prev}_prediction_map.png"), bbox_inches="tight", dpi=150)
        plt.close()

        # PROBABILITY 
        # Apply morphological gap closing
        #curr_ann_map_probs = cv2.morphologyEx(curr_ann_map_probs, cv2.MORPH_CLOSE, kernel)
        # Save the accumulated annotation matrix for the previous WSI 
        all_ann_probs_maps.append(curr_ann_map_probs) 
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = plt.get_cmap('viridis')  # continuous colormap
        im = plt.imshow(curr_ann_map_probs, cmap=cmap, interpolation='nearest')
        ax.set_title(f'Probability Map for WSI {prev}')
        ax.axis('off')
        # Shrink the colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="1.5%", pad=0.03)
        plt.colorbar(im, cax=cax, label='Probability')
        print(np.unique(curr_ann_map_probs, return_counts=True))
        plt.savefig(os.path.join(save_dir_probs, f"{prev}_probability_map.png"), bbox_inches="tight", dpi=150)
        plt.close() 

        # Start next WSI
        width, height = dimensions_dict[curr]
        width = width // patch_size 
        height = height // patch_size 
        curr_ann_map_binary = np.zeros((height, width), dtype=np.uint8) 
        curr_ann_map_probs = np.zeros((height, width), dtype=np.float16) 
        curr_ann_map_labels = np.zeros((height, width), dtype=np.uint8) 

    # Patch assignment
    y_start, y_end = y_pixels[i], min(y_pixels[i] + patch_size, curr_ann_map_binary.shape[0])
    x_start, x_end = x_pixels[i], min(x_pixels[i] + patch_size, curr_ann_map_binary.shape[1])
    patch_height = y_end - y_pixels[i]
    patch_width = x_end - x_pixels[i]

    # Directly fill value without creating a full patch array 
    y_start = y_start // patch_size  
    x_start = x_start // patch_size  
    if (y_start < height) and (x_start < width):
        curr_ann_map_binary[y_start, x_start] = y_pred_test[i] 
        curr_ann_map_probs[y_start, x_start] = y_probs[i] 
        curr_ann_map_labels[y_start, x_start] = y_true_test[i] 
        
    prev = curr

# BINARY 
# Save the last WSI
curr_ann_map_binary = cv2.morphologyEx(curr_ann_map_binary, cv2.MORPH_CLOSE, kernel)
all_ann_binary_maps.append(curr_ann_map_binary) 
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.get_cmap('PuRd', num_classes)
im = ax.imshow(curr_ann_map_binary, cmap=cmap, interpolation='nearest', vmin=0, vmax=num_classes - 1)
ax.set_title(f'Prediction Map for WSI {prev}')
ax.axis('off')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="1.5%", pad=0.03)
plt.colorbar(im, cax=cax, ticks=range(num_classes), label='Class Index')
plt.savefig(os.path.join(save_dir_binary, f"{prev}_prediction_map.png"), bbox_inches="tight", dpi=150)
plt.close() 

# PROBABILITY 
# Save the last WSI
#curr_ann_map_probs = cv2.morphologyEx(curr_ann_map_probs, cv2.MORPH_CLOSE, kernel)
all_ann_probs_maps.append(curr_ann_map_probs) 
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.get_cmap('viridis')  # continuous colormap
im = plt.imshow(curr_ann_map_probs, cmap=cmap, interpolation='nearest')
ax.set_title(f'Probability Map for WSI {prev}')
ax.axis('off')
# Shrink the colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="1.5%", pad=0.03)
plt.colorbar(im, cax=cax, label='Probability')
print(np.unique(curr_ann_map_probs, return_counts=True))
plt.savefig(os.path.join(save_dir_probs, f"{prev}_probability_map.png"), bbox_inches="tight", dpi=150)
plt.close() 

**CLUSTERING** 

In [ ]:
from sklearn.cluster import DBSCAN

patch_clusters = defaultdict(list)  # cluster_id → list of patch paths 
wsi_ids = list(wsi_metrics_dict.keys()) 

for i, binary_mask in enumerate(all_ann_binary_maps):
    
    wsi_id = wsi_ids[i] 
    
    dir_path = '/dir' 

    h, w = binary_mask.shape

    coords = []  # list of (row, col) of tumor patches

    for row in range(h):
        for col in range(w):
            if binary_mask[row, col]:  # tumor region
                coords.append([row, col])

    if len(coords) == 0:
        print(f"{wsi_id}: No tumor patches detected.")
        continue  # Skip to next WSI

    coords = np.array(coords)

    # Fit DBSCAN
    db = DBSCAN(eps=1.9, min_samples=7).fit(coords)
    labels = db.labels_

    # Count size of each cluster (excluding noise)
    label_counts = Counter(labels)
    # Filter clusters that have >= 18 patches
    valid_labels = {label for label, count in label_counts.items() if label != -1 and count >= 20}

    # Assign consistent colors only to valid clusters
    np.random.seed(42)  
    label_colors = {
        label: np.random.randint(0, 255, size=3)
        for label in valid_labels
    }

    print(f"{wsi_id}:")
    print("  Number of valid clusters:", len(valid_labels))
    print("  Noise points:", list(labels).count(-1))

    for (coord, label) in zip(coords, labels):
        if label in valid_labels:
            row, col = coord
            x = col * patch_size
            y = row * patch_size
            patch_filename = f"0{wsi_id}_{x}_{y}.jpg" if len(str(wsi_id)) == 2 else f"{wsi_id}_{x}_{y}.jpg" 
            patch_path = os.path.join(dir_path, patch_filename)
            patch_clusters[(wsi_id, label)].append(patch_path)


**SAVING CLUSTERS** 

In [ ]:
# To check if any path is mssing

for cluster_key, patch_list in patch_clusters.items(): 
    for path in patch_list:
        if not os.path.exists(path):
            print("Missing path: ", path)

In [ ]:
# Save clusters info to dataframe

rows = []
for (wsi_id, cluster_id), patch_list in patch_clusters.items():
    for patch_path in patch_list:
        rows.append({
            "WSI No.": wsi_id,
            "Cluster Id": cluster_id,
            "Patch Path": patch_path
        })

clusters_df = pd.DataFrame(rows) 
clusters_df.head(2) 

In [ ]:
clusters_df.to_csv('clusters_info_her2.csv', index = False) 